In [1]:
from liaisons_experiments.logger import setup_logger

setup_logger()

In [2]:
from liaisons_experiments.experiments.multi_experiment import ExperimentConfig, MultiExperiment
from tqdm.notebook import tqdm

exp_configs = [
    ExperimentConfig(
        model="gemma:2b"
    ),
    ExperimentConfig(
        model="gemma:7b"
    )
]

exps = MultiExperiment(exp_configs, tqdm=tqdm)

In [3]:
results = exps.run_from_csv(".dev/ibm_claim_stance_binary_sample.csv", relation_dim="binary")

predict argument relation - model=gemma:2b relation_dimension=binary:   0%|          | 0/110 [00:00<?, ?it/s]

predict argument relation - model=gemma:7b relation_dimension=binary:   0%|          | 0/110 [00:00<?, ?it/s]

In [4]:
results.f1_scores

,support,attack,macro
gemma:2b,0.609524,0.643478,0.626501
gemma:7b,0.815385,0.733333,0.774359
